### Applies previously calculated v-uv.

Zeroes out unvoiced sections of each algorithm's note estimates

In [ ]:
import numpy as np
import os

target_algos = ['yin']#, 'maddox', 'noll', 'hermes', 'crepe', 'boersma']
target_song = 'Mi Re Sotsodali_'
data_dir = '/Akamai/voice/data/pitches/'
vuv_dir = '/Akamai/voice/data/pitches-raw/'
res_dir = '/Akamai/voice/data/pitches-vuv/'

In [ ]:
def separate(adir):
    conv={}
    conv[0] = lambda s: float(s.strip() or 0)
    x,y = np.loadtxt(adir, unpack=True, usecols=(0,1), converters=conv)
    return (x,y)

In [ ]:
def apply_detection (uy, ay):
    uy = uy[:ay.size]
    ay[uy == 0] = 0
    return ay
    
def save_song (path, x, y):
    fout = open(path, "w+")
    for i in range(x.size):
        fout.write(f"{x[i]} {y[i]}\n")
    fout.close()

In [ ]:
ignore_files = ['raw.txt', 'bass', 'middle', 'top']

def ignore(name):
    for suffix in ignore_files:
        if suffix in location:
            return True
    return False

def file_existence(path, force=False):
    if force:
        return False
    return os.path.isfile(path)

In [ ]:
for algorithm in sorted(os.listdir(data_dir)):
    if algorithm in target_algos:
        for collection in sorted(os.listdir(f"{data_dir}{algorithm}")):
            for song in sorted(os.listdir(f"{data_dir}{algorithm}/{collection}")):
                if (target_song != song) and (target_song != ""):
                    continue
                for location in sorted(os.listdir(f"{data_dir}{algorithm}/{collection}/{song}")):
                    if ignore(location):
                        continue
                    if not os.path.isfile(os.path.join(vuv_dir, 'v-uv', collection, song, location)):
                        continue
                    #print(song, location)
                    #if not '009' in location:
                    #    continue
                    # add force=True to overwrite the pitch in res_dir
#                     if file_existence(f"{res_dir}{algorithm}/{collection}/{song}/{location}"):
#                         print(f"{algorithm}:{location} already exists")
#                         continue
                        
                    print(algorithm, location)
                    ux, uy = separate(f"{vuv_dir}{'v-uv'}/{collection}/{song}/{location}")
                    ax, ay = separate(f"{data_dir}{algorithm}/{collection}/{song}/{location}")
                    ay = apply_detection(uy, ay)

                    if not os.path.isdir(f"{res_dir}{algorithm}/{collection}/{song}"):
                        os.mkdir(f"{res_dir}{algorithm}/{collection}/{song}")
                    save_song(f"{res_dir}{algorithm}/{collection}/{song}/{location}", ax, ay)
                    